# ChatBot using TensorFlow with RNN and LSTM

### Steps followed:
1. Import the data from conversation.txt file
2. Process the data into a suitable format
3. Build the model
4. Train the model
5. Test the model

In [1]:
import pandas as pd
import numpy as np
import json
import nltk
import tensorflow as tf
import re
import time
import string
from sklearn.model_selection import train_test_split

## 1. Import the data

In [2]:

with open("conversation.txt", "r") as ins:
    sentences = []
    for line in ins:
        sentences.append(line)
    

## 2 Preprocess the data
The data is currently in a format that is not helpful for deep learning model. Hence the below steps are followed:

- Create a list of questions whose corresponding answers.
- Create buckets based on the length of the sentenses and tokens for padding
- Convert these questions and answers into integers

In [3]:


questions = []
answers = []
for j in range(512):
    for i in range(len(sentences)):
        if i%2 == 0:
            questions.append(sentences[i])
        else:
            answers.append(sentences[i])

In [4]:
limit = 5
for i in range(limit, limit+5):
    print(questions[i])
    print(answers[i])
    print()

how are you

I'm doing good


what's your name

I'm nameless


what can you do

anything that you want


when can I check in

the date specified on your reservation


when can I check out

whenever you want to




### 2.1 Cleaning the text
Clean the text. That is to remove punctuation and convert english word abbreviations. 

In [5]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm","i am", text)
    text = re.sub(r"he's","he is", text)
    text = re.sub(r"she's","she is", text)
    text = re.sub(r"that's","that is", text)
    text = re.sub(r"what's","what is", text)
    text = re.sub(r"where's","where is", text)
    text = re.sub(r"it's","it is", text)
    text = re.sub(r"\'ll"," will", text)
    text = re.sub(r"\'ve"," have", text)
    text = re.sub(r"\'re"," are", text)
    text = re.sub(r"\'d"," would", text)
    text = re.sub(r"won't","will not", text)
    text = re.sub(r"can't","cannot", text)
    text = re.sub(r'[-()\"#/@;:<>{}+=~|.?,!]','',text)
    return text

In [6]:
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question)) 
    
clean_answers = []
for answer in answers:
    clean_answers.append(clean_text(answer)) 

In [7]:
limit = 5
for i in range(limit, limit+5):
    print(clean_questions[i])
    print(clean_answers[i])
    print()

how are you

i am doing good


what is your name

i am nameless


what can you do

anything that you want


when can i check in

the date specified on your reservation


when can i check out

whenever you want to




In [8]:
word2count = {}
for question in clean_questions:
    for word in question.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1

for answer in clean_answers:
    for word in answer.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1

In [9]:
threshold = 0
questionswords2int = {}
word_number = 0
for word, count in word2count.items():
    if count >= threshold:
        questionswords2int[word] = word_number
        word_number += 1
        
answerswords2int = {}
word_number = 0
for word, count in word2count.items():
    if count >= threshold:
        answerswords2int[word] = word_number
        word_number += 1

### 2.2.1 Adding tokens
Now add the tokens to this dictionary. These will come in handy later on when we manipulate our strings further for the model.
We use a few special symbols to fill in the sequence.
1.	EOS : End of sentence
2.	PAD : Filler
3.	SOS : Start decoding
4.	OUT : Unknown; word not in vocabulary


In [10]:
tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']

for token in tokens:
    questionswords2int[token] = len(questionswords2int) + 1
    
for token in tokens:
    answerswords2int[token] = len(answerswords2int) + 1

In [11]:

answersints2word = {w_i: w for w, w_i in answerswords2int.items()}

In [12]:
# Add EOS at the end of each answer

for i in range(len(clean_answers)):
    clean_answers[i] += " <EOS>"

### 2.3 Translating the words to integers
As stated at the beginning of this section, translate all of the words into integers that correspond to that word.

In [13]:
questions_into_int = []
for question in clean_questions:
    ints = []
    for word in question.split():
        if word not in questionswords2int:
            ints.append(questionswords2int['<OUT>'])
        else:
            ints.append(questionswords2int[word])
    questions_into_int.append(ints)
answers_into_int = []
for answer in clean_answers:
    ints = []
    for word in answer.split():
        if word not in answerswords2int:
            ints.append(answerswords2int['<OUT>'])
        else:
            ints.append(answerswords2int[word])
    answers_into_int.append(ints)

In [14]:
limit = 5
for i in range(limit, limit+5):
    print(questions_into_int[i])
    print(answers_into_int[i])
    print()

[7, 8, 9]
[15, 61, 62, 63, 113]

[3, 4, 10, 11]
[15, 61, 64, 113]

[3, 12, 9, 13]
[65, 66, 9, 67, 113]

[14, 12, 15, 16, 17]
[48, 68, 69, 70, 10, 71, 113]

[14, 12, 15, 16, 18]
[72, 9, 67, 28, 113]



In [15]:
sorted_clean_questions = []
sorted_clean_answers = []

for length in range(1, 25 + 1):
    for i in enumerate(questions_into_int):
        if len(i[1]) == length:
            sorted_clean_questions.append(questions_into_int[i[0]])
            sorted_clean_answers.append(answers_into_int[i[0]])

## 3 Building the model
The pre-processing of the data is now over. Now create a model to take this data and learn from it.

Process for creating this model will be:
1. Define model inputs
2. Target preprocessing function
3. Create the Encoder RNN layer
4. Create Decoder RNN layer 
4. Define the Seq2Seq model

### 3.1 Model inputs function
Define the placeholders for model.

In [16]:
def model_inputs():
    inputs = tf.placeholder(tf.int32, [None, None], name = 'input')
    targets = tf.placeholder(tf.int32, [None, None], name = 'target')
    lr = tf.placeholder(tf.float32, name = 'learning_rate')
    keep_prob = tf.placeholder(tf.float32, name = 'keep_prob')
    return inputs, targets, lr, keep_prob

### 3.2 Preprocess targets
This function will process  targets in a format for model to understand, including by adding SOS (Start of sentence) at the beginning of each sentence.

In [17]:
def preprocess_targets(targets, word2int, batch_size):
    left_side = tf.fill([batch_size, 1], word2int['<SOS>'])
    right_side = tf.strided_slice(targets, [0,0], [batch_size, -1], [1,1])
    preprocessed_targets = tf.concat([left_side, right_side], 1)
    return preprocessed_targets

### 3.3 Creating the Encoder RNN layer

In [18]:
def encoder_rnn(rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length):
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
    encoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout] * num_layers)
    encoder_output, encoder_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = encoder_cell,
                                                                    cell_bw = encoder_cell,
                                                                    sequence_length = sequence_length,
                                                                    inputs = rnn_inputs,
                                                                    dtype = tf.float32)
    return encoder_state

### 3.4 Create the Decoder RNN Layer 
This part of the model is fairly involved as I need to split this section into 3 parts to create decoder RNN layer:
1. Decode the training set
2. Decode the validation set
3. Decode the RNN Layer

In [19]:
# Decoding the training set
def decode_training_set(encoder_state, decoder_cell, decoder_embedded_input, sequence_length, decoding_scope, output_function, keep_prob, batch_size):
    attention_states = tf.zeros([batch_size, 1, decoder_cell.output_size])
    attention_keys, attention_values, attention_score_function, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states, attention_option = "bahdanau", num_units = decoder_cell.output_size)
    training_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_train(encoder_state[0],
                                                                              attention_keys,
                                                                              attention_values,
                                                                              attention_score_function,
                                                                              attention_construct_function,
                                                                              name = "attn_dec_train")
    decoder_output, decoder_final_state, decoder_final_context_state = tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell,
                                                                                                              training_decoder_function,
                                                                                                              decoder_embedded_input,
                                                                                                              sequence_length,
                                                                                                              scope = decoding_scope)
    decoder_output_dropout = tf.nn.dropout(decoder_output, keep_prob)
    return output_function(decoder_output_dropout)

In [20]:
# Decoding the test/validation set
def decode_test_set(encoder_state, decoder_cell, decoder_embeddings_matrix, sos_id, eos_id, maximum_length, num_words, decoding_scope, output_function, keep_prob, batch_size):
    attention_states = tf.zeros([batch_size, 1, decoder_cell.output_size])
    attention_keys, attention_values, attention_score_function, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states, attention_option = "bahdanau", num_units = decoder_cell.output_size)
    test_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_inference(output_function,
                                                                              encoder_state[0],
                                                                              attention_keys,
                                                                              attention_values,
                                                                              attention_score_function,
                                                                              attention_construct_function,
                                                                              decoder_embeddings_matrix,
                                                                              sos_id,
                                                                              eos_id,
                                                                              maximum_length,
                                                                              num_words,
                                                                              name = "attn_dec_inf")
    test_predictions, decoder_final_state, decoder_final_context_state = tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell,
                                                                                                                test_decoder_function,
                                                                                                                scope = decoding_scope)
    return test_predictions

In [21]:
# Create the Decoder RNN
def decoder_rnn(decoder_embedded_input, decoder_embeddings_matrix, encoder_state, num_words, sequence_length, rnn_size, num_layers, word2int, keep_prob, batch_size):
    with tf.variable_scope("decoding") as decoding_scope:
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
        decoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout] * num_layers)
        weights = tf.truncated_normal_initializer(stddev = 0.1)
        biases = tf.zeros_initializer()
        output_function = lambda x: tf.contrib.layers.fully_connected(x,
                                                                      num_words,
                                                                      None,
                                                                      scope = decoding_scope,
                                                                      weights_initializer = weights,
                                                                      biases_initializer = biases)
        training_predictions = decode_training_set(encoder_state,
                                                   decoder_cell,
                                                   decoder_embedded_input,
                                                   sequence_length,
                                                   decoding_scope,
                                                   output_function,
                                                   keep_prob,
                                                   batch_size)
        decoding_scope.reuse_variables()
        test_predictions = decode_test_set(encoder_state,
                                           decoder_cell,
                                           decoder_embeddings_matrix,
                                           word2int['<SOS>'],
                                           word2int['<EOS>'],
                                           sequence_length - 1,
                                           num_words,
                                           decoding_scope,
                                           output_function,
                                           keep_prob,
                                           batch_size)
    return training_predictions, test_predictions

### 3.5 Defining the seq2seq model

In [22]:
def seq2seq_model(inputs, targets, keep_prob, batch_size, sequence_length, answers_num_words, questions_num_words, encoder_embedding_size, decoder_embedding_size, rnn_size, num_layers, questionswords2int):
    encoder_embedded_input = tf.contrib.layers.embed_sequence(inputs,
                                                              answers_num_words + 1,
                                                              encoder_embedding_size,
                                                              initializer = tf.random_uniform_initializer(0, 1))
    encoder_state = encoder_rnn(encoder_embedded_input, rnn_size, num_layers, keep_prob, sequence_length)
    preprocessed_targets = preprocess_targets(targets, questionswords2int, batch_size)
    decoder_embeddings_matrix = tf.Variable(tf.random_uniform([questions_num_words + 1, decoder_embedding_size], 0, 1))
    decoder_embedded_input = tf.nn.embedding_lookup(decoder_embeddings_matrix, preprocessed_targets)
    training_predictions, test_predictions = decoder_rnn(decoder_embedded_input,
                                                         decoder_embeddings_matrix,
                                                         encoder_state,
                                                         questions_num_words,
                                                         sequence_length,
                                                         rnn_size,
                                                         num_layers,
                                                         questionswords2int,
                                                         keep_prob,
                                                         batch_size)
    return training_predictions, test_predictions

## 4. Training the model
Our RNN model has been built, now it's time to train the model on data.

Process for training the model will be as follows:
1. Initial set-up
2. Define loss error, optimizer etc.
3. Further string manipulation
4. Training for loop

### 4.1 Initial set-up
To set up model training the following are done:
- Define hyperparameters
- Define session
- Load the model inputs
- Set length of sequence and the shape of  inputs
- Get training and test predictions

### 4.1.1 Setting hyperparameters
Training a deep learning model takes time. I had to make a pay-off between optimal chatbot performance and training time. To strike a good balance I decided on the following parameters which resulted in a training time of roughly 4 hours. 

In [23]:
epochs = 10
batch_size = 128
rnn_size = 512
num_layers = 2
encoding_embedding_size = 512
decoding_embedding_size = 512
learning_rate = 0.01
learning_rate_decay = 0.9
min_learning_rate= 0.0001
keep_probability = 0.5

In [24]:
# Define session
tf.reset_default_graph()
session = tf.InteractiveSession()

In [25]:
# Loading model inputs
inputs, targets, lr, keep_prob = model_inputs()

In [26]:
# Setting sequence length
sequence_length = tf.placeholder_with_default(25, None, name = 'sequence_length')

In [27]:
# Setting the shape of inputs tensor
input_shape = tf.shape(inputs)

In [28]:
# Getting training and test predictions
training_predictions, test_predictions = seq2seq_model(tf.reverse(inputs, [-1]),
                                                       targets,
                                                       keep_prob,
                                                       batch_size,
                                                       sequence_length,
                                                       len(answerswords2int),
                                                       len(questionswords2int),
                                                       encoding_embedding_size,
                                                       decoding_embedding_size,
                                                       rnn_size,
                                                       num_layers,
                                                       questionswords2int)

### 4.2 Defining loss error, optimizer etc.

In [29]:
with tf.name_scope("optimization"):
    loss_error = tf.contrib.seq2seq.sequence_loss(training_predictions,
                                                  targets,
                                                  tf.ones([input_shape[0], sequence_length]))
    optimizer = tf.train.AdamOptimizer(learning_rate)
    gradients = optimizer.compute_gradients(loss_error)
    clipped_gradients = [(tf.clip_by_value(grad_tensor, -5., 5.), grad_variable) for grad_tensor, grad_variable in gradients if grad_tensor is not None]
    optimizer_gradient_clipping = optimizer.apply_gradients(clipped_gradients)

### 4.3 Further string manipulation
Before entering the training for loop some further string manipulationa are done. Namely:

- Add padding so each sentence is the same length
- Split  data into the batches that will be fed into the model
- Train, dev, test split

In [35]:
# Padding the sequences with PAD
def apply_padding(batch_of_sequences, word2int):
    max_sequence_length = max([len(sequence) for sequence in batch_of_sequences])
    return [sequence + [word2int['<PAD>']] * (max_sequence_length - len(sequence)) for sequence in batch_of_sequences]

In [31]:
# Split the data into batches of questions and answers
def split_into_batches(questions, answers, batch_size):
    for batch_index in range(0, len(questions) // batch_size):
        start_index = batch_index * batch_size
        questions_in_batch = questions[start_index : start_index + batch_size]
        answers_in_batch = answers[start_index : start_index + batch_size]
        padded_questions_in_batch = np.array(apply_padding(questions_in_batch, questionswords2int))
        padded_answers_in_batch = np.array(apply_padding(answers_in_batch, answerswords2int))
        yield padded_questions_in_batch, padded_answers_in_batch

In [32]:
questions_shuffled, questions_shuffled_testing, answers_shuffled, answers_shuffled_testing = train_test_split(sorted_clean_questions,sorted_clean_answers,test_size=0.1,shuffle=True,random_state=101)


training_validation_split = int(len(questions_shuffled) * 0.1)
training_questions = questions_shuffled[training_validation_split:]
training_answers = answers_shuffled[training_validation_split:]
validation_questions = questions_shuffled[:training_validation_split]
validation_answers = answers_shuffled[:training_validation_split]

### 4.4 Training for loop
This is the section where the model will learn data.

In [36]:
# Training
batch_index_check_training_loss = 100
batch_index_check_validation_loss = ((len(training_questions)) // batch_size // 2) - 1
total_training_loss_error = 0
list_validation_loss_error = []
early_stopping_check = 0
early_stopping_stop = 100
checkpoint = "chatbot_weights.ckpt"
session.run(tf.global_variables_initializer())
for epoch in range(1, epochs + 1):
    for batch_index, (padded_questions_in_batch, padded_answers_in_batch) in enumerate(split_into_batches(training_questions, training_answers, batch_size)):
        starting_time = time.time()
        _, batch_training_loss_error = session.run([optimizer_gradient_clipping, loss_error], {inputs: padded_questions_in_batch,
                                                                                               targets: padded_answers_in_batch,
                                                                                               lr: learning_rate,
                                                                                               sequence_length: padded_answers_in_batch.shape[1],
                                                                                               keep_prob: keep_probability})
        total_training_loss_error += batch_training_loss_error
        ending_time = time.time()
        batch_time = ending_time - starting_time
        if batch_index % batch_index_check_training_loss == 0:
            print('Epoch: {:>3}/{}, Batch: {:>4}/{}, Training Loss Error: {:>6.3f}, Training Time on 100 Batches: {:d} seconds'.format(epoch,
                                                                                                                                       epochs,
                                                                                                                                       batch_index,
                                                                                                                                       len(training_questions) // batch_size,
                                                                                                                                       total_training_loss_error / batch_index_check_training_loss,
                                                                                                                                       int(batch_time * batch_index_check_training_loss)))
            total_training_loss_error = 0
        if batch_index % batch_index_check_validation_loss == 0 and batch_index > 0:
            total_validation_loss_error = 0
            starting_time = time.time()
            for batch_index_validation, (padded_questions_in_batch, padded_answers_in_batch) in enumerate(split_into_batches(validation_questions, validation_answers, batch_size)):
                batch_validation_loss_error = session.run(loss_error, {inputs: padded_questions_in_batch,
                                                                       targets: padded_answers_in_batch,
                                                                       lr: learning_rate,
                                                                       sequence_length: padded_answers_in_batch.shape[1],
                                                                       keep_prob: 1})
                total_validation_loss_error += batch_validation_loss_error
            ending_time = time.time()
            batch_time = ending_time - starting_time
            average_validation_loss_error = total_validation_loss_error / (len(validation_questions) / batch_size)
            print('Validation Loss Error: {:>6.3f}, Batch Validation Time: {:d} seconds'.format(average_validation_loss_error, int(batch_time)))
            learning_rate *= learning_rate_decay
            if learning_rate < min_learning_rate:
                learning_rate = min_learning_rate
            list_validation_loss_error.append(average_validation_loss_error)
            if average_validation_loss_error <= min(list_validation_loss_error):
                print('I speak better now!!')
                early_stopping_check = 0
                saver = tf.train.Saver()
                saver.save(session, checkpoint)
            else:
                print("Sorry, I do not speak better, I need to get trained more.")
                early_stopping_check += 1
                if early_stopping_check == early_stopping_stop:
                    break
    if early_stopping_check == early_stopping_stop:
        print("I cannot speak better anymore. This is the best I can do.")
        break
print("Game Over")

Epoch:   1/10, Batch:    0/93, Training Loss Error:  0.048, Training Time on 100 Batches: 178 seconds
Validation Loss Error:  1.237, Batch Validation Time: 5 seconds
I speak better now!!
Validation Loss Error:  0.876, Batch Validation Time: 5 seconds
I speak better now!!
Epoch:   2/10, Batch:    0/93, Training Loss Error:  3.222, Training Time on 100 Batches: 176 seconds
Validation Loss Error:  0.498, Batch Validation Time: 5 seconds
I speak better now!!
Validation Loss Error:  0.300, Batch Validation Time: 5 seconds
I speak better now!!
Epoch:   3/10, Batch:    0/93, Training Loss Error:  0.609, Training Time on 100 Batches: 179 seconds
Validation Loss Error:  0.212, Batch Validation Time: 5 seconds
I speak better now!!
Validation Loss Error:  0.189, Batch Validation Time: 5 seconds
I speak better now!!
Epoch:   4/10, Batch:    0/93, Training Loss Error:  0.280, Training Time on 100 Batches: 177 seconds
Validation Loss Error:  0.183, Batch Validation Time: 5 seconds
I speak better now

## 5. Testing chatbot
 model has now run through the training and is ready to be implemented. This is a short section and will go through this process:

1. Begin the session
2. Convert question to integers
3. Setting up the chat

In [37]:
# Loading the weights and Running the session
checkpoint = "./chatbot_weights.ckpt"
session = tf.InteractiveSession()
session.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.restore(session, checkpoint)

In [38]:
# Converting the questions to integers
def convert_string2int(question, word2int):
    question = clean_text(question)
    return [word2int.get(word, word2int['<OUT>']) for word in question.split()]

In [ ]:
# Setting up the chat
i=0
while(True):
    question = input("You: ")
    if question == 'Goodbye':
        break
    question = convert_string2int(question, questionswords2int)
    question = question + [questionswords2int['<PAD>']] * (12 - len(question))
    fake_batch = np.zeros((batch_size, 12))
    fake_batch[0] = question
    
    predicted_answer = session.run(test_predictions, {inputs: fake_batch, keep_prob: 0.7})[0]
    answer = ''
    for i in np.argmax(predicted_answer, 1):
        if answersints2word[i] == 'i':
            token = ' I'
        elif answersints2word[i] == '<EOS>':
            token = '.'
        elif answersints2word[i] == '<OUT>':
            token = 'out'
        else:
            token = ' ' + answersints2word[i]
        answer += token
        if token == '.':
            break
    print('ChatBot: ' + answer)
    i=i+1

You: hi
ChatBot:  hi.
You: how are you
ChatBot:  I am doing good.
You: who are you
ChatBot:  I am nameless.
You: what can you do
ChatBot:  I don't have phone number either.
You: what is your address
ChatBot:  I don't have an address.
You: when can I check out
ChatBot:  I don't have phone number either.
You: can you call someone for me
ChatBot:  sure I am getting you someone to talk with you.
You: I need to make an appointment
ChatBot:  sure about what.
You: I need to see the doctor
ChatBot:  sure about what.
You: My child needs to come in for a check-up
ChatBot:  can you come to the clinic.
You: When is the doctor free
ChatBot:  sure for what.
You: how to get to your office if I come from San Jose
ChatBot:  you need to take the vta.
You: how to get to your office if I come by car
ChatBot:  you need to take the vta.
You: how to get to your office if I come by motorbike
ChatBot:  you need to take the vta.
You: GoodBye
ChatBot:  have a nice day.
